"Genes are commonly represented in computer software as a sequence of the characters A, C, G, and T. Each letter represents a nucleotide, and the combination of three nucleotides is called a codon... A codon codes for a specific amino acid that together with other amino acids can form a protein. A classic task in bioinformatics software is to find a particular codon within a gene. "

Using __IntEnum__ to represent a nucleotide

for more about __enum__ see : https://docs.python.org/3/library/enum.html#enum.IntEnum

The type __IntEnum__ allows us to compare the values with comparison operations (<,>, = and so on).

In [1]:
from enum import IntEnum
from typing import Tuple, List

Nucleotide: IntEnum = IntEnum('Nucleotide', ('A','C','G','T'))

A codon has 3 Nucleotides. A gene consists of a list of condons:

In [2]:
Codon = Tuple[Nucleotide, Nucleotide, Nucleotide]
Gene = List[Codon]

The __string_to_gene()__ function goes through the gene_str, converting every 3 characters into a __Codon__, which is in turn added to the end of a new __Gene__. When there are no __Nucleotide__ 's two places in the future of the current i it is examining, the last one or two nucleotides are ignored as incomplete genes. 

In [3]:
gene_str: str = 'ACGTGGCTCTCTAACGTACGTACGTACGGGGTTTATATATACCCTAGGACTCCCTTT'

def string_to_gene(s: str) -> Gene:
    gene: Gene = []
    for i in range(0, len(s), 3):
        if (i + 2) >= len(s):
            return gene
        codon: Codon = (Nucleotide[s[i]], Nucleotide[s[i + 1]], Nucleotide[s[i + 2]])
        gene.append(codon) # add condon to gene
    return gene

In [4]:
my_gene: Gene = string_to_gene(gene_str)

In [5]:
my_gene

[(<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.G: 3>),
 (<Nucleotide.T: 4>, <Nucleotide.G: 3>, <Nucleotide.G: 3>),
 (<Nucleotide.C: 2>, <Nucleotide.T: 4>, <Nucleotide.C: 2>),
 (<Nucleotide.T: 4>, <Nucleotide.C: 2>, <Nucleotide.T: 4>),
 (<Nucleotide.A: 1>, <Nucleotide.A: 1>, <Nucleotide.C: 2>),
 (<Nucleotide.G: 3>, <Nucleotide.T: 4>, <Nucleotide.A: 1>),
 (<Nucleotide.C: 2>, <Nucleotide.G: 3>, <Nucleotide.T: 4>),
 (<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.G: 3>),
 (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.C: 2>),
 (<Nucleotide.G: 3>, <Nucleotide.G: 3>, <Nucleotide.G: 3>),
 (<Nucleotide.G: 3>, <Nucleotide.T: 4>, <Nucleotide.T: 4>),
 (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.T: 4>),
 (<Nucleotide.A: 1>, <Nucleotide.T: 4>, <Nucleotide.A: 1>),
 (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.C: 2>),
 (<Nucleotide.C: 2>, <Nucleotide.C: 2>, <Nucleotide.T: 4>),
 (<Nucleotide.A: 1>, <Nucleotide.G: 3>, <Nucleotide.G: 3>),
 (<Nucleotide.A: 1>, <Nucleotide.C: 2>, 

### Searching for a particular codon - Linear Search

To search for a particular codon, we are going to use a technique called Linear search. In the worst case, a linear search will go through every single element in the data structure. It has a complexity of O(n).

In [6]:
def linear_contains(gene: Gene, key_codon: Codon) -> bool:
    for codon in gene:
        if codon == key_codon:
            return True
    return False

In [7]:
acg: Codon = (Nucleotide.A, Nucleotide.C, Nucleotide.G)
gat: Codon = (Nucleotide.G, Nucleotide.A, Nucleotide.T)
print(linear_contains(my_gene, acg))  # True
print(linear_contains(my_gene, gat))  # False

True
False


Note:
"This function is for illustrative purposes only. The Python built-in sequence types (list, tuple, range) all implement the __ contains__() method, which allows us to do a search for a specific item in them by simply using the in operator. In fact, the in operator can be used with any type that implements __ contains__(). For instance, we could search my_gene for acg and print out the result by writing print(acg in my_gene)."

In [8]:
acg in my_gene

True

### Binary Search

A binary search works by checking the middle element of a sorted range of elements, comparing it to the element we are looking for. Based on this comparison, it will know whether the element we are looking for is left or right of this middle element. This process repeats until the element is found.

The search space is reduced by half at every step, so it has a worst case runtime of O(lg n). Sorting itself requires O(n lg n) time for the best sorting algorithms. So binary search only makes sense if we are going to perform searches many times.

A binary search for a codon:

In [9]:
def binary_contains(gene: Gene, key_codon: Codon) -> bool:
    low: int = 0
    high: int = len(gene)-1
    while low <= high: # while there is still a search space
        mid: int = (low + high) // 2
        if gene[mid] < key_codon:
            low = mid + 1
        elif gene[mid] > key_codon:
            high = mid - 1
        else:
            return True
    return False

Before we run our gene through our function, we need to sort first:

In [10]:
my_sorted_gene: Gene = sorted(my_gene) #gene is a list
print(binary_contains(my_sorted_gene, acg))
print(binary_contains(my_sorted_gene, gat))

True
False


note: there is a module called __bisect__ in Python's standard library for building binary searches.

### Generic Search code:

In [11]:
from __future__ import annotations
from typing import TypeVar, Iterable, Sequence, Generic, List, Callable, Set, Deque, Dict, Any, Optional
from typing_extensions import Protocol
from heapq import heappush, heappop
#importing all sorts of data types available in Python

T = TypeVar('T')
def linear_contains(iterable: Iterable[T], key: T) -> bool:
    for item in iterable:
        if item == key:
            return True
    return False

C = TypeVar('C', bound = 'Comparable')

class Comparable(Protocol):
    def __eq__(self, other: Any)-> bool:
        pass
    
    def __lt__(self: C, other: C) -> bool:
        pass
    
    def __gt__(self: C, other: C) -> bool:
        return (not self < other) and self != other
    
    def __le__(self: C, other: C) -> bool:
        return self < other or self == other
    
    def __ge__(self: C, other: C) -> bool:
        return not self < Other
    
def binary_contains(sequence: Sequence[C], key: C) -> bool:
    low: int = 0
    high: int = len(sequence) - 1
    while low <= high:  # while there is still a search space
        mid: int = (low + high) // 2
        if sequence[mid] < key:
            low = mid + 1
        elif sequence[mid] > key:
            high = mid - 1
        else:
            return True
    return False

In [12]:
print(linear_contains([1, 5, 15, 15, 15, 15, 20], 5))
print(binary_contains(['a', 'd', 'e', 'f', 'z'], 'f'))
print(binary_contains(['john', 'mark', 'ronald', 'sarah'], 'sheila'))

True
True
False
